# Collecting all N-block to N-block diffs

Example: N=2

All 2-block to 2-block diffs are collected here

A 2-block to 2-block diff is a diff that contains line or block, that in-place modifies to a different line/block of code (or gets deleted or added).

We can represent one such combination as:
a->b
c->d

where a,b,c,d could be 0,1,2,3,4 where 0 means there was no code before. 1,2,3,4 are identifiers for a specific content of a line/block.

The standard diff format operations are:
- 0->a (added line/block)
- a->0 (deleted line/block)
- a->b (modified line/block)

We can rule out some unimportant cases:
- No-operations: 0->0 (nothing there -> nothing there)
- Independent operations:
    - a->b
    - c->d
    where a and b are different from c and d.
    this means that the operations do not influence each other or are related.

Special cases:
Identical line/blocks: (1->1, 2->2, 3->3, 4->4)


In [97]:
# Iterate through all combinations for N-block to N-block diffs
# a->b
# c->d
# e->f
# ....
# So create a 2*N-tuple of (a,b,c,d) with possible values for each: (0,1,...,L)
# So for N=1 and L=1, we have the standard git diff format:
# 0->1
# ----
# 1->0
# For N=2 and L=1 we have:
# 0->1
# 0->1
# ----
# 1->0
# 0->1
# ----
# etc.

In [98]:
import itertools

def canonicalize(tup):
    # Change (3,2,2,5) to (1,2,2,3) since the labels don't matter
    # But 0 stays 0.
    # More examples:
    #   (0,0,3,0) -> (0,0,1,0)
    #   (4,3,2,1) -> (1,2,3,4)
    #   (2,3,1,4) -> (1,2,3,4)
    #   (3,2,0,4) -> (1,2,0,3)
    tup_map = {0:0}
    next_idx = 1
    for x in range(len(tup)):
        if tup[x] in tup_map:
            # It's already remapped, e.g. 0 stays 0.
            continue
        else:
            tup_map[tup[x]] = next_idx
            next_idx += 1
    # Now remap using the tup_map
    return tuple([tup_map[x] for x in tup])

def ignore_subtuple_order(tup):
    # Now also sort the tuple such that the order of tuple pairs is ignored
    # so (tup[0], tup[1]) <= (tup[2], tup[3]) <= (tup[4], tup[5]) <= ...
    pairs = [(tup[ii], tup[ii+1]) for ii in range(0,len(tup),2)]
    sorted_pairs = sorted(pairs)
    return tuple([x for pair in sorted_pairs for x in pair])
    return tup

    
def is_uninteresting(tup):
    # The following cases are uninteresting:
    # No-operations
    # Independent operations:
    # Identical line/blocks

    # 0->0 is a no-op
    for ii in range(0,len(tup),2):
        if tup[ii] == 0 and tup[ii+1] == 0:
            return True


    # 1->2
    # 3->4 are independent
    for ii in range(0,len(tup),2):
        for jj in range(0,len(tup),2):
            if ii==jj:
                continue

            if ((tup[ii] == 0 or (
                tup[ii] != tup[jj] 
                and tup[ii]!=tup[jj+1] )
                )
                and (tup[ii+1] == 0 or (
                    tup[ii+1] != tup[jj] 
                    and tup[ii+1]!=tup[jj+1]))):
                return True

    # Check if one of the line/blocks stay the same
    for ii in range(0,len(tup),2):
        if tup[ii] == tup[ii+1]:
            return True



In [99]:

L = 1 # number of labels and 0
N = 3 # number of line/blocks being considered
# Note: L>2*N is not useful


# Create list [0,1,...,L]
L_eff = min(max(L+1,1),2*N)
vals = list(range(L_eff))

# N = 1, return (0,0), (0,1), ... ,(L,L)
# N = 2, return (0,0,0,0), (0,0,0,1), ... ,(L,L,L,L)
all_combs = list(itertools.product(vals, repeat=2*N))


# Remove all combinations up to relabelling of 1,2,3,4
canonical_combs = list(set(ignore_subtuple_order(canonicalize(x)) for x in all_combs))
canonical_combs = sorted(canonical_combs)
poss_combs = [x for x in canonical_combs if not is_uninteresting(x)]


print(len(all_combs))
print(len(canonical_combs))
print(len(poss_combs))

64
20
4


In [100]:
for ii,comb in enumerate(poss_combs):
    print(f"Combination {ii+1}/{len(poss_combs)}")
    for jj in range(0,len(comb),2):
        print(f"{comb[jj]} -> {comb[jj+1]}")
    print("-------")

Combination 1/4
0 -> 1
0 -> 1
0 -> 1
-------
Combination 2/4
0 -> 1
0 -> 1
1 -> 0
-------
Combination 3/4
0 -> 1
1 -> 0
1 -> 0
-------
Combination 4/4
1 -> 0
1 -> 0
1 -> 0
-------
